In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
df = pd.read_csv("../data/x_train.csv")
df_val = pd.read_csv("../data/x_train.csv")

In [2]:
from importlib import import_module
import sklearn

# Testing models

In [3]:
y_train = df['FINALIZED_LOAN']
X_train = df.drop(columns="FINALIZED_LOAN")

# Validation data
y_val = df_val['FINALIZED_LOAN']
X_val = df_val.drop(columns="FINALIZED_LOAN")


In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
scores = cross_validate(model, X_train, y_train, cv= 6, n_jobs=1, scoring="f1")
scores

{'fit_time': array([2.99905372, 3.09530139, 3.15768623, 2.89740801, 3.26939511,
        3.07994175]),
 'score_time': array([0.09541726, 0.10302281, 0.09031129, 0.08708096, 0.10857773,
        0.10220408]),
 'test_score': array([0.62516915, 0.60962567, 0.63433814, 0.61559508, 0.67729084,
        0.62087912])}

In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
# Initialize stratified k-fold
kf = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)

# Initialize a variable to keep track of fold number
fold = 1

# Iterate over each fold
for train_index, test_index in kf.split(X_train, y_train):
    print('Fold {} of {}'.format(fold, kf.n_splits))
    
    # Split data into train and validation sets
    xtr, xvl = X_train.iloc[train_index], X_train.iloc[test_index]
    ytr, yvl = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Initialize and train the decision tree classifier
    model = DecisionTreeClassifier(random_state=1)
    model.fit(xtr, ytr)
    
    # Make predictions on the validation set
    pred_val = model.predict(xvl)
    
    # Calculate and print accuracy score
    score = accuracy_score(yvl, pred_val)
    print('Accuracy Score:', score)
    
    # Increment fold number
    fold += 1


Fold 1 of 5
Accuracy Score: 0.859271523178808
Fold 2 of 5
Accuracy Score: 0.8367549668874172
Fold 3 of 5
Accuracy Score: 0.8595561444186817
Fold 4 of 5
Accuracy Score: 0.8598873799271282
Fold 5 of 5
Accuracy Score: 0.8575687313680026


In [6]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train the random forest classifier on the entire training set
rf_model = RandomForestClassifier(class_weight='balanced', random_state=1)
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
rf_pred_val = rf_model.predict(X_val)

# Calculate accuracy score on the validation set
rf_val_score = accuracy_score(y_val, rf_pred_val)
print('Random Forest Validation Accuracy Score:', rf_val_score)


Random Forest Validation Accuracy Score: 0.9997350466980195


In [7]:
# Initialize and train the decision tree classifier on the entire training set
model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)

# Make predictions on the validation set
pred_val = model.predict(X_val)

# Calculate accuracy score on the validation set
val_score = accuracy_score(y_val, pred_val)
print('Validation Accuracy Score:', val_score)


Validation Accuracy Score: 0.9997350466980195


In [8]:
# Initialize and train the decision tree classifier on the entire training set
model = DecisionTreeClassifier(random_state=1)
model.fit(X_train, y_train)

# Make predictions on the validation set
pred_val = model.predict(X_val)

# Calculate accuracy score on the validation set
val_score = accuracy_score(y_val, pred_val)
print('Validation Accuracy Score:', val_score)


SyntaxError: invalid syntax (3474560347.py, line 9)

In [9]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the logistic regression classifier on the entire training set
logistic_model = LogisticRegression(random_state=1)
logistic_model.fit(X_train, y_train)

# Make predictions on the validation set
logistic_pred_val = logistic_model.predict(X_val)

# Calculate accuracy score on the validation set
logistic_val_score = accuracy_score(y_val, logistic_pred_val)
print('Logistic Regression Validation Accuracy Score:', logistic_val_score)


Logistic Regression Validation Accuracy Score: 0.8527522024243227


C:\Users\Sebastian\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [10]:
import xgboost as xgb

# Convert the DataFrame into DMatrix format (required for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Set hyperparameters for XGBoost
params = {
    'objective': 'binary:logistic',  # Use binary classification
    'eval_metric': 'error',           # Evaluate using classification error
    'seed': 1                         # Set random seed for reproducibility
}

# Train the XGBoost model
num_rounds = 436  # Number of boosting rounds (you can tune this hyperparameter)
xgb_model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the validation set
xgb_pred_val = xgb_model.predict(dval)
xgb_pred_val_binary = [1 if p > 0.5 else 0 for p in xgb_pred_val]

# Calculate accuracy score on the validation set
xgb_val_score = accuracy_score(y_val, xgb_pred_val_binary)
print('XGBoost Validation Accuracy Score:', xgb_val_score)

ModuleNotFoundError: No module named 'xgboost'

In [11]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Generate confusion matrix
xgb_conf_matrix = confusion_matrix(y_val, xgb_pred_val_binary)
xgb_conf_matrix
# Plot confusion matrix
# plt.figure(figsize=(8, 6))
# sns.heatmap(xgb_conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
# plt.xlabel('Predicted labels')
# plt.ylabel('True labels')
# plt.title('XGBoost Confusion Matrix')
# plt.show()


NameError: name 'xgb_pred_val_binary' is not defined

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Initialize and train the Gradient Boosting Classifier on the entire training set
gb_model = GradientBoostingClassifier(random_state=1)
gb_model.fit(X_train, y_train)

# Make predictions on the validation set
gb_pred_val = gb_model.predict(X_val)

# Calculate accuracy score on the validation set
gb_val_score = accuracy_score(y_val, gb_pred_val)
print('Gradient Boosting Classifier Validation Accuracy Score:', gb_val_score)


Gradient Boosting Classifier Validation Accuracy Score: 0.8883884215407034


In [13]:
# Training data
y_train = df['FINALIZED_LOAN']
X_train = df.drop(columns="FINALIZED_LOAN")

In [14]:
# Initialize stratified k-fold
kf = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Initialize a variable to keep track of fold number
fold = 1

# Iterate over each fold
for train_index, test_index in kf.split(X_train, y_train):
    print('Fold {} of {}'.format(fold, kf.n_splits))
    
    # Split data into train and validation sets
    xtr, xvl = X_train.iloc[train_index], X_train.iloc[test_index]
    ytr, yvl = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Initialize and train the decision tree classifier
    model = RandomForestClassifier(random_state=1)
    model.fit(xtr, ytr)
    
    # Make predictions on the validation set
    pred_val = model.predict(xvl)
    
    # Calculate and print accuracy score
    score = accuracy_score(yvl, pred_val)
    print('Accuracy Score:', score)
    
    # Increment fold number
    fold += 1


Fold 1 of 10
Accuracy Score: 0.8894039735099337
Fold 2 of 10
Accuracy Score: 0.8960264900662251
Fold 3 of 10
Accuracy Score: 0.8834437086092716
Fold 4 of 10
Accuracy Score: 0.895364238410596
Fold 5 of 10
Accuracy Score: 0.9052980132450331
Fold 6 of 10
Accuracy Score: 0.8913907284768212
Fold 7 of 10
Accuracy Score: 0.890728476821192
Fold 8 of 10
Accuracy Score: 0.8979456593770709
Fold 9 of 10
Accuracy Score: 0.8899933730947648
Fold 10 of 10
Accuracy Score: 0.8906560636182903


In [15]:
# Initialize stratified k-fold
kf = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)

# Initialize a variable to keep track of fold number
fold = 1

# Iterate over each fold
for train_index, test_index in kf.split(X_train, y_train):
    print('Fold {} of {}'.format(fold, kf.n_splits))
    
    # Split data into train and validation sets
    xtr, xvl = X_train.iloc[train_index], X_train.iloc[test_index]
    ytr, yvl = y_train.iloc[train_index], y_train.iloc[test_index]
    
    # Initialize and train the decision tree classifier
    model = RandomForestClassifier(random_state=1)
    model.fit(xtr, ytr)
    
    # Make predictions on the validation set
    pred_val = model.predict(xvl)
    
    # Calculate and print accuracy score
    score = f1_score(yvl, pred_val)
    print('Accuracy Score:', score)
    
    # Increment fold number
    fold += 1

Fold 1 of 10
Accuracy Score: 0.6178489702517163
Fold 2 of 10
Accuracy Score: 0.642369020501139
Fold 3 of 10
Accuracy Score: 0.6088888888888889
Fold 4 of 10
Accuracy Score: 0.6565217391304348
Fold 5 of 10
Accuracy Score: 0.6786516853932585
Fold 6 of 10
Accuracy Score: 0.645021645021645
Fold 7 of 10
Accuracy Score: 0.6325167037861915
Fold 8 of 10
Accuracy Score: 0.6515837104072398
Fold 9 of 10
Accuracy Score: 0.6261261261261261
Fold 10 of 10
Accuracy Score: 0.6275395033860045
